In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-ql5oeh78
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-ql5oeh78
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done


In [14]:
import os
import pandas as pd
import torch
import clip


In [15]:
import random
random.seed(10)

In [16]:
%cd '/content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project'
%ls
path = os.getcwd()
print('path: ' + path)
OutputPath = path + '/Output'
ModelPath = path + '/Models'
DataPath = path + '/Data'

/content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project
 Chat_GPT_rating_accuracy.ipynb   GPT2_prediction.ipynb   Output/
 CNN_rating_prediction.ipynb      LLaVa_Inference.ipynb   Prediction_with_CLIP.ipynb
 Data/                            Models/                'Score comparision.ipynb'
path: /content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project


In [17]:
# Paths
llava_inference_image_only_path = OutputPath + '/llava_inference_images.csv'  # Inference file (images only)
llava_inference_text_image_path = OutputPath + '/llava_inference_text_images.csv'  # Inference file (text + images)

# Load LLAVA inference data
llava_inference_images = pd.read_csv(llava_inference_image_only_path)
llava_inference_text_images = pd.read_csv(llava_inference_text_image_path)

In [18]:
# Load review data and filter by 'taste' category
df = pd.read_csv(DataPath + '/reviews.csv')
df_of_tastes = df[df['rating_category'] == 'taste'].copy()
df_of_tastes.reset_index(drop=True, inplace=True)

# Reset indices for merging
df_of_tastes.reset_index(inplace=True)
llava_inference_images.reset_index(inplace=True)
llava_inference_text_images.reset_index(inplace=True)

# Merge actual ratings from df_of_tastes
llava_inference_images = llava_inference_images.merge(df_of_tastes[['index', 'rating']], on='index', how='inner')
llava_inference_text_images = llava_inference_text_images.merge(df_of_tastes[['index', 'rating']], on='index', how='inner')

In [19]:
model, preprocess = clip.load("ViT-B/32", device="cuda")
# Create rating features
rating_texts = ["Rating 1", "Rating 2", "Rating 3", "Rating 4", "Rating 5"]
rating_tokens = clip.tokenize(rating_texts).to('cuda')
#Generate rating features from tex
with torch.no_grad():
    rating_features = model.encode_text(rating_tokens)

In [20]:
def predict_clip_rating(llava_text):
    # Truncate the text to fit within CLIP's context length (77 tokens)
    max_length = 77  # CLIP context length
    truncated_text = llava_text[:max_length]

    # Tokenize and encode truncated LLAVA text
    text_input = clip.tokenize([truncated_text]).to('cuda')
    with torch.no_grad():
        text_features = model.encode_text(text_input)

    # Calculate cosine similarities
    similarities = (text_features @ rating_features.T).softmax(dim=-1)

    # Return the index of the highest similarity as the predicted rating (1-5)
    predicted_rating = similarities.argmax(dim=-1).item() + 1
    return predicted_rating

# Predict ratings for LLAVA text inferences with truncation applied
llava_inference_text_images['predicted_rating'] = llava_inference_text_images['inference'].apply(predict_clip_rating)

# Save the predictions along with actual ratings to a CSV file
output_file = os.path.join(OutputPath, 'clip_predictions_text_images_with_actual_ratings.csv')
llava_inference_text_images.to_csv(output_file, index=False)

print(f"Predictions saved to {output_file}")

Predictions saved to /content/drive/My Drive/Colab Notebooks/Deep learning/LLaVa_food_grade_project/Output/clip_predictions_text_images_with_actual_ratings.csv
